In [18]:
import pandas as pd

from pathlib import Path

from star_parser import StarParser, StarTabDf

In [19]:
work_folder = Path(r"/mnt/DATA/xiaohu/20220301_Necen_PRC11/cryosparc/P140/exports/jobs/P140_J10_extract_micrographs_multi/P140_J10_particles")
star_in=work_folder/"P140_J10_particles_exported.star"
star_in.exists()

True

In [20]:
parser = StarParser(star_in)
data = parser.parse()
ptcls = data["data_particles"]
ptcls.to_df()


,MicrographName,CoordinateX,CoordinateY
0,FoilHole_302096_Data_328764_328766_20220302_00...,3900,2844
1,FoilHole_302096_Data_328764_328766_20220302_00...,1020,624
2,FoilHole_302096_Data_328764_328766_20220302_00...,2624,2620
3,FoilHole_302096_Data_328764_328766_20220302_00...,1760,1936
4,FoilHole_302096_Data_328764_328766_20220302_00...,2656,1528
...,...,...,...
1457639,FoilHole_319308_Data_328764_328766_20220301_14...,5420,176
1457640,FoilHole_319308_Data_328764_328766_20220301_14...,836,1480
1457641,FoilHole_319308_Data_328764_328766_20220301_14...,1284,1728
1457642,FoilHole_319308_Data_328764_328766_20220301_14...,4836,464


In [21]:
ptcls.keep_only_columns(['MicrographName','CoordinateX','CoordinateY'], store=True)

,MicrographName,CoordinateX,CoordinateY
0,FoilHole_302096_Data_328764_328766_20220302_00...,3900,2844
1,FoilHole_302096_Data_328764_328766_20220302_00...,1020,624
2,FoilHole_302096_Data_328764_328766_20220302_00...,2624,2620
3,FoilHole_302096_Data_328764_328766_20220302_00...,1760,1936
4,FoilHole_302096_Data_328764_328766_20220302_00...,2656,1528
...,...,...,...
1457639,FoilHole_319308_Data_328764_328766_20220301_14...,5420,176
1457640,FoilHole_319308_Data_328764_328766_20220301_14...,836,1480
1457641,FoilHole_319308_Data_328764_328766_20220301_14...,1284,1728
1457642,FoilHole_319308_Data_328764_328766_20220301_14...,4836,464


In [22]:
prefix = ">J4/motioncorrected/"
ptcls.remove_string_from_column_name(prefix, 'MicrographName', store=True)

,MicrographName,CoordinateX,CoordinateY
0,FoilHole_302096_Data_328764_328766_20220302_00...,3900,2844
1,FoilHole_302096_Data_328764_328766_20220302_00...,1020,624
2,FoilHole_302096_Data_328764_328766_20220302_00...,2624,2620
3,FoilHole_302096_Data_328764_328766_20220302_00...,1760,1936
4,FoilHole_302096_Data_328764_328766_20220302_00...,2656,1528
...,...,...,...
1457639,FoilHole_319308_Data_328764_328766_20220301_14...,5420,176
1457640,FoilHole_319308_Data_328764_328766_20220301_14...,836,1480
1457641,FoilHole_319308_Data_328764_328766_20220301_14...,1284,1728
1457642,FoilHole_319308_Data_328764_328766_20220301_14...,4836,464


In [23]:
ptcls.trim_column_values('MicrographName', start=len('000004137433332187832_'), store=True)


,MicrographName,CoordinateX,CoordinateY
0,28764_328766_20220302_005914_fractions.mrc,3900,2844
1,28764_328766_20220302_005914_fractions.mrc,1020,624
2,28764_328766_20220302_005914_fractions.mrc,2624,2620
3,28764_328766_20220302_005914_fractions.mrc,1760,1936
4,28764_328766_20220302_005914_fractions.mrc,2656,1528
...,...,...,...
1457639,28764_328766_20220301_145815_fractions.mrc,5420,176
1457640,28764_328766_20220301_145815_fractions.mrc,836,1480
1457641,28764_328766_20220301_145815_fractions.mrc,1284,1728
1457642,28764_328766_20220301_145815_fractions.mrc,4836,464


In [24]:
#check that all micrographs have been properly edited. They typically start with the same prefix (specify below)
[p for p in ptcls.to_df()['MicrographName'] if not p.startswith('Foil')]

['28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_328766_20220302_005914_fractions.mrc',
 '28764_32876

In [25]:
remove_cryosparc_suffix = True
if remove_cryosparc_suffix:
    ptcls.remove_string_from_column_name('_patch_aligned_doseweighted', 'MicrographName', store=True)
ptcls.to_df()['MicrographName'][0]

In [ ]:
ptcls.add_prefix_to_column('MotionCorr/job002/Micrographs/', 'MicrographName', store=True)

,MicrographName,CoordinateX,CoordinateY
0,MotionCorr/job002/Micrographs/FoilHole_302096_...,3900,2844
1,MotionCorr/job002/Micrographs/FoilHole_302096_...,1020,624
2,MotionCorr/job002/Micrographs/FoilHole_302096_...,2624,2620
3,MotionCorr/job002/Micrographs/FoilHole_302096_...,1760,1936
4,MotionCorr/job002/Micrographs/FoilHole_302096_...,2656,1528
...,...,...,...
1457639,MotionCorr/job002/Micrographs/FoilHole_319308_...,5420,176
1457640,MotionCorr/job002/Micrographs/FoilHole_319308_...,836,1480
1457641,MotionCorr/job002/Micrographs/FoilHole_319308_...,1284,1728
1457642,MotionCorr/job002/Micrographs/FoilHole_319308_...,4836,464


In [17]:
out = work_folder/'particles.star'
data.write_out(to_file=True, new_file=out)

AttributeError: 'DataFrame' object has no attribute 'write_out'

In [13]:
split = ptcls.to_df().groupby('MicrographName')
split_dir = work_folder/'split'
if not split_dir.exists():
    split_dir.mkdir()
for fname, d in split:
    x = StarTabDf(d)
    fname = fname.replace('.mrc','.star')
    with open(split_dir/fname, 'w') as f:
        f.write(x.to_star())


In [ ]:
#if we want to train cryolo, we need to remove the data_ and loop_ lines from the header
cryolo_train = False
starfiles = split_dir.glob("*.star")
if cryolo_train:
    for f in starfiles:
        text = f.read_text().split('\n')
        for index, line in enumerate(text):
            if line.startswith('_rln'):
                break
        f.write_text('\n'.join(text[index:]))



In [23]:
data['data_optics'].to_df()

,Voltage,SphericalAberration,AmplitudeContrast,OpticsGroup,ImageSize,ImagePixelSize,ImageDimensionality
0,300.000000,2.700000,0.100000,1,200,2.122000,2
